In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision import datasets,transforms,models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import random
import cv2
import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error as rmse
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import albumentations as A
import tqdm
from tqdm import trange,tqdm
from time import sleep

In [2]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
tidcol = test['Id']
idcol = train['Id']
direct1 = '../input/petfinder-pawpularity-score/train'
for i in range(idcol.shape[0]):
    idcol[i] = os.path.join(direct1, f'{idcol[i]}.jpg')
direct = '../input/petfinder-pawpularity-score/test'
for i in range(tidcol.shape[0]):
    tidcol[i] = os.path.join(direct, f'{tidcol[i]}.jpg')


In [3]:
class Pawpularity(Dataset):
    def __init__(self,path,targets=None,transform = None):
        self.path = path
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self,idx):
        ipath = self.path[idx]
        image = Image.open(ipath)
        #image = cv2.imread(ipath)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #conv = transforms.ToTensor()
        #image = conv(image)
        #image = torch.rot90(image,1,[1,2])
        image = np.array(image)
        if self.transform is not None:
            al_dict = self.transform(image=image)
        #print(image.size)
        image = al_dict['image']
        conv = transforms.ToTensor()
        image = conv(image)
        image = torch.rot90(image,1,[1,2])
        if self.targets is not None :
            label = torch.tensor(self.targets[idx]).float()
            return image,label
        else :
            return image

In [4]:
transform = A.Compose([
    A.SmallestMaxSize(240),
    A.RandomCrop(width=224 , height=224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.518, 0.484, 0.446), std=(0.264, 0.260, 0.261))
])

In [5]:
paths = train['Id']
target = train['Pawpularity']
tpaths = test['Id']
cutedata = Pawpularity(path  = paths.values,targets = target , transform = transform)
testdata = Pawpularity(path = tpaths.values,targets = None , transform = None )

In [6]:
def imageplot(dataset = cutedata, count =3):
    plt.figure(figsize = (20,20))
    for i in range(count):
        rand = random.randint(0, len(dataset))
        image,label = dataset[rand]
        plt.subplot(1, count, i%count +1)
        plt.axis('off')
        plt.imshow(image.permute(2, 1, 0))
        plt.title(f'Pawpularity: {label}')

In [7]:
for i in range(4):
    imageplot(count=4)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if(torch.cuda.is_available()):
  print("The device is " + torch.cuda.get_device_name())
else:
    print("Using cpu no gpu available")
print('\n',device)

In [9]:
class kfoldtrainer:
    def __init__(self,model,loss_func,optimizer , scheduler,dataset,device,epochs = 1,kfolds = 5):
        self.model = model
        self.epochs = epochs
        self.kfolds = kfolds
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.dataset = dataset
        self.loss_func = loss_func
    def reset(self,model):
        for layers in model.children():
            if hasattr(layer, 'reset_parameters') and layer.requires_grad == True :
                layer.reset_parameters()
            
    def trainer(self):
        kfold = KFold(n_splits=self.kfolds, shuffle=True)
        for folds,(trainind,testind) in enumerate(kfold.split(self.dataset)):
            print(f'FOLD {folds}')
            print('--------------------------------')
            train_subsampler = torch.utils.data.SubsetRandomSampler(trainind)
            test_subsampler = torch.utils.data.SubsetRandomSampler(testind)
            trainloader = torch.utils.data.DataLoader(
                      self.dataset, 
                      batch_size=64, sampler=train_subsampler)
            testloader = torch.utils.data.DataLoader(
                              self.dataset,
                              batch_size=64, sampler=test_subsampler)
            #self.model.apply(reset)
            self.model.to(device)
            iters=[]
            J = []
            i = 0
            for epoch in range(1,self.epochs+1):
                print(f'Starting epoch {epoch}')
                current_loss = 0.0
                self.model = self.model.train()
                scores = []
                targets = []
                with tqdm(trainloader, unit="batch") as tepoch:
                    for images,target in tepoch:
                        tepoch.set_description(f"Epoch {epoch}")
                        images = images.to(device)
                        target = target.to(device)
                        score = self.model(images)
                        a,b = score.shape
                        target = torch.reshape(target,(a,1))
                        loss = self.loss_func(score,target)
                        targets.append(target.detach().cpu().numpy())
                        scores.append(score.detach().cpu().numpy())
                        J.append(float(loss))
                        iters.append(i)
                        i+=1
                        self.optimizer.zero_grad()
                        loss.backward()
                        self.optimizer.step()
                        tepoch.set_postfix(loss=loss.item())
                        sleep(0.1)
                        #if not bno % 50:
                           # print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                            #       %(epoch+1, epochs, bno, 
                             #        len(train_loader), float(loss)))
                self.scheduler.step()
                print(self.optimizer.state_dict()['param_groups'][0]['lr'])
                self.model = self.model.eval()
                with torch.set_grad_enabled(False): 
                    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
                          epoch, self.epochs, 
                        rmse(np.concatenate(targets,axis = 0) , np.concatenate(scores,axis = 0),squared = False)))
            plt.plot(iters,J) 
            plt.show()
        
                    
            
 
            
        
    

In [10]:
!pip install timm
!pip install torchinfo
from torchinfo import summary
import timm

In [11]:
all_densenet_models = timm.list_models('*efficientnet*')
all_densenet_models

In [12]:
efb4 = timm.create_model('efficientnet_b4', pretrained=True)

In [13]:
metric = nn.MSELoss()
#optimizer = torch.optim.Adam(densenet.parameters(),lr = 1e-3)

    
num_ftrs = efb4.classifier.in_features
efb4.classifier = nn.Sequential(
           nn.Linear(num_ftrs,128),
           nn.ReLU(inplace = True),
           nn.Linear(128, 1))
optimizer1 = torch.optim.Adam(efb4.parameters(),lr = 1e-2)
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=5,gamma=0.1)

print("Done")

In [14]:
summary(efb4, input_size=(32, 3, 224,224))

In [15]:
kfolder = kfoldtrainer(efb4,metric,optimizer1,scheduler1,cutedata,device,epochs = 15)

In [16]:
kfolder.trainer()

In [17]:
torch.cuda.empty_cache()